##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# TensorFlow Hub ile Metin Siniflandirmasi: Film yorumlari

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


Bu kitapcikta film yorumlarini *olumlu* ve *olumsuz* olarak siniflandiracagiz. Bu makine ogrenmesinde cok kullanilan onemli bir problemin, *ikili* ya da iki-parcali-siniflandirmanin bir ornegidir.

TensorFlow Hub ve Keras ile ogrenim aktarmasinin basit bir kullanimini bu egitimde gorecegiz.

[Internet Movie Database](https://www.imdb.com/) sitesinden alinan 50,000 film yorumu metinini iceren [IMDB veri setini](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) kullanacagiz. Bunlarin 25,000 adetini egitim icin ve geri kalan 25,000 adetini ise testler icin ayiralim. Egitim ve test setleri ayni sayida olumlu ve olumsuz yorumlar icerdikleri icin *dengelenmislerdir*. 

Bu kitapcik yuksek-seviye API olan [tf.keras](https://www.tensorflow.org/guide/keras) kullanarak modelleri TensorFlow ile olusturur ve egitir, ve ogrenim aktarmasi icin [TensorFlow Hub](https://www.tensorflow.org/hub) kitapligini ve platformunu kullanacagiz. `tf.keras` kullanan daha ileri bir metin siniflandirmasi egitim kitapcigi icin lutfen [MLCC Metin Siniflandirmasi Kilavuzua bakiniz](https://developers.google.com/machine-learning/guides/text-classification/).

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  # %tensorflow_version sadece Colab icinde bulunur.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

## IMDB veri setini indirelim

IMDB veri setini [imdb yorumlari](https://github.com/tensorflow/datasets/blob/master/docs/datasets.md#imdb_reviews) ya da [TensorFlow veri setlerinde](https://github.com/tensorflow/datasets) bulabilirsiniz. Asagidaki kod IMDB veri setini makinenize (ya da colab calisma zamanina) indirir:

In [0]:
# Egitim setini %60 ve %40 olarak ayiralim. Boylece 15,000 ornek egitim icin,
# 10,000 ornek dogrulama icin ve 25,000 ornek ise test etmek icin kullanilabilir.
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

## Veriyi inceleyelim 

Verinin yapisini anlamak icin bir goz atalim. Her ornek bir film yorumuna ait bir cumle ve etiketten olusur. Cumle herhangibir sekilde onceden islenmemistir. Etiket ise olumsuz yorumlar icin 0 veya olumlu yorumlar icin 1 olarak belirlenmistir.

Ilk 10 ornegi ekrana yazdiralim.

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

Ilk 10 etiketi de yazdiralim.

In [0]:
train_labels_batch

## Modeli olusturalim

Sinir agi katmanlarin ust uste konmasiyla olusturulur. Bu asamada uc ana mimari karar vermemiz gerekir:

* Metini nasil temsil edecegiz?
* Modelimizde kac katman kullanacagiz?
* Her katman icin kac adet *sakli birim* kullanacagiz?

Bu ornekte giris verisi cumlelerden olusur. Etiketler ise 0 ya da 1 idir.

Metinleri temsil etmenin bir yolu cumleleri gomulu yoneylere cevirmektir. Onceden egitilmis metin gomulmesini ilk katmaninmiz olarak kullanabiliriz, ki bunun uc avantaji vardir:
*   metin islenmesi icin kaygilanmamiza gerek kalmaz,
*   ogrenim aktarimindan yararlanabiliriz,
*   gomulmenin boyutu sabit oldugu icin islenmesi daha basittir.

Bu ornekte [TensorFlow Hub](https://www.tensorflow.org/hub) icinde bulunan [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1) adindaki *onceden egitilmis bir metin gommesi modelini* kullanacagiz.

Sectigimiz modele benzeyen uc adet daha onceden egitilmis model bulunmaktadir:

* [google/tf2-preview/gnews-swivel-20dim-with-oov/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1) - [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1) ile ayni yapidadir, fakat sozlukteki kelimeler %2.5i, 00V kovalarina donusturulmustur. Eger modelinizin sozlugu ile kullanacaginiz metnin sozlugu birbiriyle ortusmezse bu donusum yardimci olabilir.
* [google/tf2-preview/nnlm-en-dim50/1](https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1) - yaklasik 1 milyon kelimelik sozlugu ve 50 boyutu ile daha buyuk bir model.
* [google/tf2-preview/nnlm-en-dim128/1](https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1) - yaklasik 1 milyon kelimelik sozlugu ve 128 boyutu ile daha da buyuk bir model.

Simdi TensorFlow Hub modeli ile cumlelerimizi gomecek bir Keras katmani olusturalim ve bunu birkac giris orneginde deneyelim. Giris metninin uzunluguna bagli olmadan, gomulme isleminden cikan butun verilerin su sekilde olduguna dikkat edelim: `(num_examples, embedding_dimension)`.

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

Simdi butun modeli olusturalim:

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Katmanlar sirayla ust uste konarak siniflandirici olusturulur:

1. Ilk katman TensorFlow Hub katmani. Bu katman onceden egitilmis Saved Model kullanarak bir cumleyi gomulu yoneyiyle esler. Bizim burda kullandigimiz model, ([google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1)), her cumleyi simgelere ayirir, bu simgeler gomer ve sonrasinda bu gommeleri bir araya getirir. Elde edilen sonuc su boyutlara sahiptir: `(num_examples, embedding_dimension)`.
2. Bu sabit uzunluktaki cikis yoneyi 16 gizli birimli tamamen bagli (`Dense`) katmana gonderilir.
3. Son katman ise tek cikis dugumlu yogun bagli bir katmandir. Sonucta olasiligi ya da guvenirlik seviyesini belirten, `sigmoid` etkinlestici fonksiyonu sonucu ortaya cika, 0 ile 1 arasinda bir reel sahi elde edilir.

Modelimizi birlestirelim.

### Kayip fonksiyonu ve en iyilestirici

Bir modelin egitimi icin bir kayip fonksiyonu ve en iyilestirici gereklidir. Elimizde ikili siniflandirma problemi ve olasilik cikartan bir model bulundugu icin, `binary_crossentropy` kayip fonksiyonunu kullanacagiz. 

Bu kayip fonksiyonu icin tek secenegimiz degil, ornegin `mean_squared_error` fonksiyonunu da secebilirdik. Fakat olasilik hesaplari soz konusu oldugunda `binary_crossentropy` fonksiyonunun genellikle daha iyi oldugunu goruyoruz. Bu fonksiyon olasilik dagilimlari arasindaki "uzakligi" olcer, ki bu bizim ornegimizde gercek deger dagilimlari ile tahminler arasindaki uzakliktir.

Daha sonra gerileme sorunlarini incelerken (bir evin degerini tahmin etme problemi gibi), bir baska kayip fonksiyonu olan 'mean squared error'u nasil kullanacagimizi gorecegiz.

Simdi modelimizi kayip fonksiyonunu ve en iyilestiriciyi kullanacak sekilde ayarlayalim:

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Modeli egitelim

512 ornekten olusan kucuk topluluklarla modelimizi 20 devir suresince egitelim. Bu butun orneklerdeki `x_train` ve `y_train` tensorlarinda 20 kere yineleme anlamina gelir. Egitim sirasinda dogrulama setindeki 10,000 ornegi kullanarak modelimizin kaybini ve dogrulugunu takip edelim:

In [0]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

## Modeli degerlendirelim

Modelimizin nasil calistigini inceleyelim. Iki degerin donduruldugunu goruyoruz. Kayip (hatamizi belirten bir sayi, ne kadar kucuk o kadar iyi) ve dogruluk.

In [0]:
results = model.evaluate(test_data.batch(512), verbose=0)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

Bu gorece basit yontemle yaklasik %87 degerinde bir dogruluk elde ettik. Daha ileri yontemlerle bu degeri %95'e cikartabiliriz.

## Baska kaynaklar

Dizgi girisleri ile calismak icin daha genel yontemler ve egitim sirasindaki dogruluk ile kaybin daha detayli analizleri icin [buraya goz atiniz](https://www.tensorflow.org/tutorials/keras/basic_text_classification).